# Technical Issues With Ollama
This Jupyter Notebook creates a GUI interface for interacting with local Ollama AI models, providing real-time streaming responses for technical questions about programming, software engineering, cybersecurity, and LLMs. The interface features model selection dropdowns, question input areas, and displays responses as properly formatted markdown with true streaming output that updates incrementally as the AI generates content. The notebook includes robust error handling, status indicators, and a technical tutoring system that connects to the Ollama API running on localhost:11434.
* **Initialization and Setup**
  * Imports necessary libraries for HTTP requests, JSON handling, and Jupyter notebook widgets
  * Sets up the Ollama API base URL (localhost:11434) and defines a system prompt for technical tutoring
  * Creates the main GUI class that will handle all user interactions
* **Model Discovery and Loading**
  * Connects to the local Ollama service to fetch all available AI models
  * Makes an API call to /api/tags endpoint to retrieve installed models
  * Handles connection errors if Ollama isn't running or models aren't available
  * Populates a dropdown menu with the discovered models for user selection
* **Question Processing and Validation**
  * Validates user input to ensure a question has been entered
  * Checks that a valid model has been selected from the dropdown
  * Prepares the API request with both the system prompt and user question
  * Disables the submit button during processing to prevent multiple simultaneous requests
* **Real-Time Streaming Response**
  * Sends a streaming request to Ollama's chat API with the selected mode
  * Processes incoming response chunks in real-time as they arrive from the AI
  * Accumulates the response content and continuously updates the display
  * Renders the response as properly formatted markdown with syntax highlighting
  * Adds small delays between updates to create a smooth streaming visual effect
* **Error Handling and Status Management**
  * Monitors for various error conditions like connection failures or API errors
  * Displays informative error messages in the output area when problems occur
  * Updates the status indicator with color-coded messages (blue for info, green for success, red for errors)
  * Always re-enables the submit button after processing completes, regardless of success or failure
* **Interactive Features**
  * Provides a refresh button to reload available models without restarting the interface
  * Includes a clear button to remove previous responses for new questions
  * Maintains a clean, organized layout with proper spacing and visual hierarchy
  * Displays the complete interface in the Jupyter notebook for immediate use

In [57]:
# Ollama Technical Tutor with True Streaming Markdown Output
# This notebook creates a GUI interface for interacting with local Ollama models
# providing real-time streaming responses for technical questions

# Standard library imports
import os          # For environment variable access (if needed in future)
import requests    # For making HTTP requests to Ollama API
import json        # For parsing JSON responses from Ollama
import time        # For adding delays in streaming output

# Jupyter notebook specific imports
import ipywidgets as widgets                              # For creating interactive GUI widgets
from IPython.display import display, clear_output, Markdown  # For displaying content and markdown

# Ollama API configuration
OLLAMA_BASE_URL = "http://localhost:11434"  # Default Ollama server URL

# System prompt for technical tutoring - defines the AI's role and behavior
SYSTEM_PROMPT = """You are a helpful technical tutor who answers questions about programming, software engineering, cybersecurity and LLMs. 
Provide clear, detailed explanations with examples when appropriate. Break down complex concepts into understandable parts 
and offer practical guidance for implementation."""

class OllamaTutorMarkdownStreamingGUI:
    """
    A GUI class for creating an interactive technical tutor using Ollama models.
    
    This class provides a complete interface for:
    - Selecting available Ollama models
    - Inputting technical questions
    - Streaming markdown responses in real-time
    - Managing conversation sessions
    
    Attributes:
        available_models (list): List of available Ollama models
        current_output: Placeholder for output management
        model_dropdown: Widget for model selection
        question_input: Widget for question input
        submit_button: Widget for submitting questions
        clear_button: Widget for clearing responses
        output_area: Widget for displaying streaming responses
        status_label: Widget for showing status messages
    """
    
    def __init__(self):
        """
        Initialize the GUI interface.
        
        Sets up the complete interface by:
        1. Loading available models from Ollama
        2. Creating all GUI widgets
        3. Arranging widgets in the layout
        """
        self.available_models = []
        self.current_output = None
        self.load_available_models()
        self.setup_widgets()
        self.setup_layout()
    
    def load_available_models(self):
        """
        Fetch available models from the local Ollama installation.
        
        Makes an API call to Ollama's /api/tags endpoint to retrieve
        all installed models. Handles connection errors gracefully
        and provides informative error messages.
        
        Updates:
            self.available_models: List of model names or error messages
        """
        try:
            # Make GET request to Ollama API to fetch available models
            response = requests.get(f"{OLLAMA_BASE_URL}/api/tags")
            if response.status_code == 200:
                models_data = response.json()
                # Extract model names from the API response
                self.available_models = [model['name'] for model in models_data.get('models', [])]
                if not self.available_models:
                    self.available_models = ["No models found - Please install models first"]
            else:
                self.available_models = ["Error connecting to Ollama"]
        except requests.exceptions.ConnectionError:
            # Handle case where Ollama service is not running
            self.available_models = ["Ollama not running - Please start Ollama service"]
        except Exception as e:
            # Handle any other unexpected errors
            self.available_models = [f"Error: {str(e)}"]
    
    def setup_widgets(self):
        """
        Create and configure all GUI widgets for the interface.
        
        Creates:
        - Model selection dropdown
        - Refresh button for updating model list
        - Text area for question input
        - Submit and clear buttons
        - Output area for streaming responses
        - Status label for user feedback
        
        Also binds event handlers to interactive widgets.
        """
        # Model selection dropdown - allows user to choose from available models
        self.model_dropdown = widgets.Dropdown(
            options=self.available_models,
            value=self.available_models[0] if self.available_models else "No models available",
            description='Model:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # Refresh button - updates the model list without restarting
        self.refresh_button = widgets.Button(
            description='Refresh Models',
            button_style='info',
            layout=widgets.Layout(width='120px')
        )
        
        # Question input area - large text field for technical questions
        self.question_input = widgets.Textarea(
            placeholder='Enter your technical question here...',
            description='Question:',
            layout=widgets.Layout(width='600px', height='100px'),
            style={'description_width': 'initial'}
        )
        
        # Submit button - processes the question and streams response
        self.submit_button = widgets.Button(
            description='Ask Question',
            button_style='success',  # Green color for primary action
            layout=widgets.Layout(width='150px')
        )
        
        # Clear button - removes current response for new questions
        self.clear_button = widgets.Button(
            description='Clear Response',
            button_style='warning',  # Orange color for secondary action
            layout=widgets.Layout(width='150px')
        )
        
        # Output area - displays streaming markdown responses
        self.output_area = widgets.Output()
        
        # Status indicator - shows current operation status with colored messages
        self.status_label = widgets.HTML(
            value="<b>Status:</b> Ready to answer your question",
            layout=widgets.Layout(margin='10px 0px')
        )
        
        # Bind event handlers to interactive widgets
        self.submit_button.on_click(self.on_submit_clicked)
        self.clear_button.on_click(self.on_clear_clicked)
        self.refresh_button.on_click(self.on_refresh_clicked)
    
    def setup_layout(self):
        """
        Arrange all widgets into a cohesive interface layout.
        
        Creates a vertical layout with:
        - Header with title and description
        - Model selection row with refresh button
        - Question input area
        - Control buttons row
        - Status indicator
        - Response output area
        """
        # Header section with title and description
        header = widgets.HTML(
            value="<h2>🦙 Ollama Technical Tutor - True Markdown Streaming</h2><p>Ask technical questions with real-time markdown streaming output</p>",
            layout=widgets.Layout(margin='0px 0px 20px 0px')
        )
        
        # Model selection row - dropdown and refresh button side by side
        model_row = widgets.HBox([
            self.model_dropdown,
            widgets.HTML(value="&nbsp;" * 3),  # Spacer for visual separation
            self.refresh_button
        ], layout=widgets.Layout(margin='10px 0px'))
        
        # Control buttons row - submit and clear buttons side by side
        controls_row = widgets.HBox([
            self.submit_button,
            widgets.HTML(value="&nbsp;" * 5),  # Spacer for visual separation
            self.clear_button
        ], layout=widgets.Layout(margin='10px 0px'))
        
        # Main interface - vertical arrangement of all components
        self.interface = widgets.VBox([
            header,
            model_row,
            self.question_input,
            controls_row,
            self.status_label,
            widgets.HTML(value="<b>Response:</b>"),
            self.output_area
        ], layout=widgets.Layout(padding='20px'))
    
    def on_submit_clicked(self, button):
        """
        Handle submit button click events.
        
        Validates user input and initiates the streaming response process.
        Performs input validation and model selection verification before
        calling the streaming response method.
        
        Args:
            button: The button widget that triggered the event (unused)
        """
        # Get and validate user question input
        question = self.question_input.value.strip()
        
        if not question:
            self.update_status("Please enter a question before submitting.", "error")
            return
        
        # Validate model selection
        selected_model = self.model_dropdown.value
        if "Error" in selected_model or "No models" in selected_model:
            self.update_status("Please select a valid model or refresh the model list.", "error")
            return
        
        # Initiate streaming response
        self.stream_markdown_response(question)
    
    def on_clear_clicked(self, button):
        """
        Handle clear button click events.
        
        Clears the output area and updates status to indicate readiness
        for new questions.
        
        Args:
            button: The button widget that triggered the event (unused)
        """
        # Clear the output display area
        with self.output_area:
            clear_output()
        self.update_status("Response cleared. Ready for a new question.", "success")
    
    def on_refresh_clicked(self, button):
        """
        Handle refresh button click events.
        
        Reloads the list of available models from Ollama and updates
        the dropdown widget with the current model list.
        
        Args:
            button: The button widget that triggered the event (unused)
        """
        self.update_status("Refreshing available models...", "info")
        # Reload models from Ollama API
        self.load_available_models()
        # Update dropdown options with new model list
        self.model_dropdown.options = self.available_models
        if self.available_models:
            self.model_dropdown.value = self.available_models[0]
        self.update_status("Models refreshed successfully!", "success")
    
    def update_status(self, message, status_type="info"):
        """
        Update the status label with colored messages.
        
        Provides visual feedback to users about the current state of operations
        using color-coded messages for different types of status updates.
        
        Args:
            message (str): The status message to display
            status_type (str): Type of status - 'info', 'success', 'error', or 'warning'
        """
        # Color mapping for different status types
        colors = {
            "info": "#2196F3",      # Blue for informational messages
            "success": "#4CAF50",   # Green for successful operations
            "error": "#f44336",     # Red for error conditions
            "warning": "#FF9800"    # Orange for warnings
        }
        
        color = colors.get(status_type, colors["info"])
        self.status_label.value = f"<b style='color: {color}'>Status:</b> {message}"
    
    def stream_markdown_response(self, question):
        """
        Stream the AI response as markdown in real-time.
        
        This is the core method that:
        1. Sends the question to Ollama API with streaming enabled
        2. Processes the streaming response chunks
        3. Updates the display with accumulated markdown content
        4. Handles errors gracefully with informative messages
        
        Args:
            question (str): The user's technical question to be answered
        """
        selected_model = self.model_dropdown.value
        
        # Update UI state for processing
        self.update_status(f"Streaming response from {selected_model}...", "info")
        self.submit_button.disabled = True  # Prevent multiple simultaneous requests
        
        # Clear previous response from output area
        with self.output_area:
            clear_output()
        
        # Prepare API request payload with system prompt and user question
        payload = {
            "model": selected_model,
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": question}
            ],
            "stream": True  # Enable streaming response
        }
        
        try:
            # Make streaming POST request to Ollama chat API
            response = requests.post(
                f"{OLLAMA_BASE_URL}/api/chat",
                json=payload,
                stream=True,
                timeout=60  # 60 second timeout for long responses
            )
            
            if response.status_code == 200:
                accumulated_response = ""  # Build complete response incrementally
                
                # Process each line of the streaming response
                for line in response.iter_lines():
                    if line:
                        try:
                            # Parse JSON chunk from streaming response
                            chunk_data = json.loads(line.decode('utf-8'))
                            
                            # Extract content from the message chunk
                            if 'message' in chunk_data and 'content' in chunk_data['message']:
                                chunk_content = chunk_data['message']['content']
                                accumulated_response += chunk_content
                                
                                # Update display with current accumulated response
                                with self.output_area:
                                    clear_output(wait=True)  # Clear previous content
                                    display(Markdown(accumulated_response))  # Display as markdown
                                
                                # Small delay for smooth visual streaming effect
                                time.sleep(0.03)
                            
                            # Check if streaming is complete
                            if chunk_data.get('done', False):
                                break
                                
                        except json.JSONDecodeError:
                            # Skip malformed JSON chunks and continue
                            continue
                
                # Update status when streaming completes successfully
                self.update_status("Response completed successfully!", "success")
            else:
                # Handle HTTP error responses from Ollama API
                error_msg = f"Ollama API error: {response.status_code}"
                self.update_status(error_msg, "error")
                
                with self.output_area:
                    clear_output()
                    display(Markdown(f"**❌ Error:** {error_msg}\n\nPlease check if Ollama is running and the model is available."))
                
        except requests.exceptions.ConnectionError:
            # Handle connection errors when Ollama service is not available
            error_msg = "Cannot connect to Ollama. Please ensure Ollama is running on localhost:11434"
            self.update_status(error_msg, "error")
            
            with self.output_area:
                clear_output()
                display(Markdown(f"**❌ Connection Error:** {error_msg}"))
                
        except Exception as e:
            # Handle any other unexpected errors during streaming
            error_msg = f"Unexpected error: {str(e)}"
            self.update_status(error_msg, "error")
            
            with self.output_area:
                clear_output()
                display(Markdown(f"**❌ Error:** {error_msg}"))
        
        finally:
            # Always re-enable the submit button when processing completes
            self.submit_button.disabled = False
    
    def display(self):
        """
        Display the complete GUI interface in the Jupyter notebook.
        
        This method renders the entire interface, making it visible and
        interactive for users. Should be called after instantiating the class.
        """
        display(self.interface)

# Create and display the Ollama technical tutor GUI
# This instantiates the class and immediately displays the interface
ollama_tutor = OllamaTutorMarkdownStreamingGUI()
ollama_tutor.display()